In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor


from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

from sklearn.model_selection import GridSearchCV

from datetime import datetime

pd.set_option('display.max_rows', 5000)

<H1>External feature integration

In [17]:
df=pd.read_csv('../data/zri_acs_bikeshare_merged.csv')

In [18]:
df['year-month']=df['datetime'].apply(lambda x: x[0:7])

In [19]:
df

,zip,zri,year,month,datetime,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,...,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing,year-month
0,1013,928.0,2014,1,2014-01-01,0.728315,0.020951,0.013732,0.219830,0.000000,...,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0,2014-01
1,1013,931.0,2014,2,2014-02-01,0.727052,0.021179,0.013708,0.220867,0.000000,...,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0,2014-02
2,1013,934.0,2014,3,2014-03-01,0.725788,0.021408,0.013685,0.221904,0.000000,...,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0,2014-03
3,1013,929.0,2014,4,2014-04-01,0.724525,0.021636,0.013661,0.222940,0.000000,...,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0,2014-04
4,1013,929.0,2014,5,2014-05-01,0.723261,0.021864,0.013638,0.223977,0.000000,...,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0,2014-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93667,99654,1277.0,2019,8,2019-08-01,0.796036,0.008872,0.014941,0.049974,0.049727,...,0.102326,0.049199,33.566667,0.475244,0.399400,0.018076,0.0,0.0,0,2019-08
93668,99654,1271.0,2019,9,2019-09-01,0.795709,0.008765,0.014871,0.050125,0.049560,...,0.102254,0.049419,33.575000,0.475511,0.399650,0.018306,0.0,0.0,0,2019-09
93669,99654,1299.0,2019,10,2019-10-01,0.795381,0.008658,0.014801,0.050275,0.049393,...,0.102181,0.049640,33.583333,0.475778,0.399900,0.018537,0.0,0.0,0,2019-10
93670,99654,1261.5,2019,11,2019-11-01,0.795054,0.008550,0.014730,0.050426,0.049226,...,0.102109,0.049861,33.591667,0.476045,0.400150,0.018767,0.0,0.0,0,2019-11


<H1>Standardization

In [20]:
# zri3['mean']=zri3['RegionName'].apply(lambda x: std_dict[x][0])
# zri3['std']=zri3['RegionName'].apply(lambda x: std_dict[x][1])

#zri3['ZRI_scaled']=(zri3['ZRI']-zri3['mean'])/zri3['std']
#zri3['ZRI_scaled']=zri3['zri']
#zri3=zri3.drop(['zri'],axis=1)
# zri3=zri3.drop(['mean','std'],axis=1)

<H1>Lag development

In [21]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [22]:
def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

In [23]:
def laggenerator_diff6(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+6}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+6)
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+6}']=0
    return df

In [24]:
def laggenerator_MA(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+4]
    
    df.loc[:,f'{colname}_MA{i}']=df.loc[:,f'{colname}'].shift(i)-(df.loc[:,f'{colname}'].shift(i)+df.loc[:,f'{colname}'].shift(i+1)+df.loc[:,f'{colname}'].shift(i+2)+df.loc[:,f'{colname}'].shift(i+3)+df.loc[:,f'{colname}'].shift(i+4))/5
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_MA{i}']=0
    return df

In [25]:
def laggenerator_mean(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_mean{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_mean{i}']+=df.loc[:,f'{colname}'].shift(x)
    df.loc[:,f'{colname}_mean{i}']=df.loc[:,f'{colname}_mean{i}']/i
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_mean{i}']=0
    return df

In [26]:
def laggenerator_ewm(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_ewm{i}']=0
    weightsum=0
    for y in range(1,i):
        weight=i-y
        weightsum+=weight
    for x in range(1,i):
        weight=i-x
        df.loc[:,f'{colname}_ewm{i}']+=df.loc[:,f'{colname}'].shift(x)*weight/weightsum
    df.loc[:,f'{colname}_ewm{i}']=df.loc[:,f'{colname}_ewm{i}']
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_ewm{i}']=0
    return df

In [27]:
def laggenerator_min(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_min{i}']=999999
    for x in range(1,i):
        df.loc[:,f'{colname}_min{i}']=np.minimum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_min{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_min{i}']=0
    return df

In [28]:
def laggenerator_max(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i-1]
    df.loc[:,f'{colname}_max{i}']=0
    for x in range(1,i):
        df.loc[:,f'{colname}_max{i}']=np.maximum(df.loc[:,f'{colname}'].shift(x),df.loc[:,f'{colname}_max{i}'])
    
    df.loc[df['year-month'].isin(timelist),f'{colname}_max{i}']=0
    return df

In [29]:
def lag_generator_all(df, var):
    for i in range(1,14):   #14
        df=laggenerator(i, var,df)

    for i in range(1,14):  #11
        df=laggenerator_diff(i, var,df)

#     for i in range(1,6):
#         df=laggenerator_diff6(i, var,df)

    #for i in range(1,11):
     #   df=laggenerator_MA(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_mean(i, 'ZRI_scaled',df)

    for i in range(1,14):
        df=laggenerator_ewm(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_min(i, var,df)

#     for i in range(1,14):
#         df=laggenerator_max(i, var,df)
    
    return df

In [30]:
df.columns[5:43]

Index(['percent_white', 'percent_black', 'percent_asian', 'percent_hispanic',
       'percent_native_am', 'percent_other_race', 'percent_0_17',
       'percent_18_39', 'percent_40_64', 'percent_65+',
       'percent_rental_units_vacant', 'percent_rental_units_occupied',
       'percent_graduate_deg', 'percent_bachelors', 'percent_associates',
       'percent_highschool', 'percent_less_highschool',
       'percent_commute_public_transport', 'percent_commute_less_30',
       'percent_commute_30_to_59', 'percent_commute_60_to_89',
       'percent_commute_90_more', 'percent_new_city', 'percent_new_unit',
       'percent_units_owner_occupied', 'median_building_age',
       'income_per_capita', 'poverty_rate', 'total_pop',
       'percent_workforce_unemployed', 'percent_work_from_home', 'median_age',
       'percent_female', 'gini_index', 'percent_not_us_citizen',
       'bs_total_stations', 'bs_total_systems', 'has_bike_sharing'],
      dtype='object')

In [31]:
for var in df.columns[5:43]:
    df=lag_generator_all(df, var)

# df=lag_generator_all(df, 'zri')   
# df=df.drop(['percent_white', 'percent_black',
#        'percent_asian', 'percent_hispanic', 'percent_native_am',
#        'percent_other_race', 'percent_0_17', 'percent_18_39', 'percent_40_64',
#        'percent_65+', 'percent_rental_units_vacant',
#        'percent_rental_units_occupied', 'percent_graduate_deg',
#        'percent_bachelors', 'percent_associates', 'percent_highschool',
#        'percent_less_highschool', 'percent_commute_public_transport',
#        'percent_commute_less_30', 'percent_commute_30_to_59',
#        'percent_commute_60_to_89', 'percent_commute_90_more',
#        'percent_new_city', 'percent_new_unit', 'percent_units_owner_occupied',
#        'median_building_age', 'income_per_capita', 'poverty_rate', 'total_pop',
#        'percent_workforce_unemployed', 'percent_work_from_home', 'median_age',
#        'percent_female', 'gini_index', 'percent_not_us_citizen',
#        'bs_total_stations', 'bs_total_systems', 'has_bike_sharing'],axis=1)

In [32]:
df.columns

Index(['zip', 'zri', 'year', 'month', 'datetime', 'percent_white',
       'percent_black', 'percent_asian', 'percent_hispanic',
       'percent_native_am',
       ...
       'has_bike_sharing_ewm4', 'has_bike_sharing_ewm5',
       'has_bike_sharing_ewm6', 'has_bike_sharing_ewm7',
       'has_bike_sharing_ewm8', 'has_bike_sharing_ewm9',
       'has_bike_sharing_ewm10', 'has_bike_sharing_ewm11',
       'has_bike_sharing_ewm12', 'has_bike_sharing_ewm13'],
      dtype='object', length=1526)

In [33]:
train=df[(df['year-month']<='2018-12') & (df['year-month']>'2015-01')]

train=train.drop('year-month',axis=1)

Y_train=train['zri']
X_train=train.drop('zri',axis=1)


In [34]:
labelencoder = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()
enc=OneHotEncoder()
enc2=OneHotEncoder()

enc2_df = pd.DataFrame(enc2.fit_transform(X_train[['zip']]).toarray())
X_train = X_train.join(enc2_df, rsuffix='1')
X_train=X_train.fillna(0)

X_train=X_train.drop(['zip','year','month','datetime'],axis=1)


In [35]:
# linmod=RandomForestRegressor(n_estimators=500)

# params=({
#      'min_samples_split':[6,8,12], 
#         'max_features':[100,200,300],
#     'min_samples_leaf':[4,6,8]
#  })

# grid=GridSearchCV(linmod, param_grid=params, cv=5)
# grid.fit(X_train, Y_train)

# grid.best_params_

In [ ]:
#kr_model=KernelRidge(kernel='chi2',gamma=0.0001)   #gamma=0.00000001

#{'degree': 3, 'gamma': 1.0000000230258512, 'kernel': 'linear'}
#kr_model=KernelRidge(kernel='linear',gamma=1, degree=3)   #gamma=0.00000001

#kr_model=KernelRidge(kernel='polynomial',gamma=1e-06,degree=3)

#kr_model=SVR(kernel='poly',gamma=0.000001,degree=3)
#kr_model=Lasso(alpha=100)   #alpha=10

kr_model=RandomForestRegressor(min_samples_split=8, max_features=0.8, n_estimators=500)

%time kr_model.fit(X_train, Y_train)

In [ ]:
kr_model.score(X_train, Y_train)

In [ ]:
#pd.DataFrame({'Feature':X_train.columns,'Coefs':kr_model.coef_}).sort_values('Coefs')

In [ ]:
Y_test=df[(df['year-month']>'2018-12') & (df['year-month']<='2019-12')][['zri']]
predictor_table=df.copy()
X_test=predictor_table[predictor_table['year-month']>'2018-12']

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    
    enc2_df2 = pd.DataFrame(enc2.transform(X_test[['zip']]).toarray())
    X_test = X_test.join(enc2_df2, rsuffix='1')
    X_test=X_test.fillna(0)    
    
    X_test=X_test.drop(['zip','year','month','datetime','year-month','zri'],axis=1)
    val=kr_model.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'zri']=val
        
    #predictor_table=lag_generator_all(predictor_table, 'zri')
    for var in predictor_table.columns[5:43]:
        predictor_table=lag_generator_all(predictor_table, var)
        
    X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [ ]:
#Y_test['ZRI_Yest_scaled']=Y_test['ZRI_scaled']
rstable=X_test[['zri','year-month']].merge(Y_test[['zri']], left_index=True, right_index=True)


In [ ]:
RSME=(sum((rstable['zri_x']-rstable['zri_y'])**2)/len(rstable))**0.5
RSME

In [ ]:
#128 (max without autoregressive)(random forest)

In [27]:
rstable['residual']=rstable['zri_x']-rstable['zri_y']

In [28]:
rstable['residual_squared']=rstable['residual'].map(lambda x: x**2)

In [29]:
rstable2=rstable.groupby('State').agg('mean').sort_values('residual_squared')
rstable2=rstable2.reset_index()

KeyError: 'State'

In [ ]:
fig = px.choropleth(rstable2,
                    locations='State',
                    color='residual_squared',
                    color_continuous_scale='Viridis',
                    hover_name='State',
                    locationmode='USA-states',
                    scope='usa')

fig.show()

In [ ]:
rstable[rstable['State']=='CA'].groupby('RegionName').agg('mean').sort_values('residual_squared')

In [ ]:
testmonths=list(zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]['year-month'].drop_duplicates())

R2_test=zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')][['ZRI_scaled','year-month']]
R2_test['ZRI_result']=0

for i,month in enumerate(testmonths):
    R2_test.loc[R2_test['year-month']==month,'ZRI_result']=list(zri3.loc[zri3['year-month']=='2018-12','ZRI_scaled'].apply(lambda x: x*(1+i*0.002)))

RSME_null=(sum((R2_test['ZRI_scaled']-R2_test['ZRI_result'])**2)/len(R2_test))**0.5
RSME_null


In [ ]:
zips=rstable['RegionName'].unique()[0:10]

In [ ]:
px.line(rstable[rstable['RegionName'].isin(zips)], y=['ZRI_scaled_x','ZRI_scaled_y'],x='year-month', color='RegionName')

In [ ]:
zips=zri3['RegionName'].unique()[0:5]
px.line(zri3[zri3['RegionName'].isin(zips)], y=['ZRI_scaled'],x='year-month', color='RegionName')